## decode data from all images in a folder and save data as .txt file with proper size (1 px)

In [ ]:
import numpy as np

def remove_65(data):
    data = np.array(data)  # Ensure input is a NumPy array
    duplicate_indices = []

    # Find consecutive duplicates in the slice
    if len(data) == 129: 
        # Find the smallest number and its index in the range 0 to 64
        min_index_0_128 = min(range(0, 128), key=lambda i: data[i])
        duplicate_indices.append(min_index_0_128) 

    if len(data) == 130:
        # Find the smallest number and its index in the range 0 to 64
        min_index_0_64 = min(range(0, 65), key=lambda i: data[i])
        duplicate_indices.append(min_index_0_64)
        # Find the smallest number and its index in the range 65 to 129
        min_index_65_129 = min(range(65, 130), key=lambda i: data[i])
        duplicate_indices.append(min_index_65_129)
        # Remove the smallest elements from both ranges
        
    if len(data) == 131:
        # Find the smallest number and its index in the range 0 to 64
        min_index_0_64 = min(range(0, 64), key=lambda i: data[i])
        duplicate_indices.append(min_index_0_64)
        # Find the smallest number and its index in the range 65 to 129
        min_index_65_129 = min(range(66, 131), key=lambda i: data[i])
        duplicate_indices.append(min_index_65_129)
        # Remove 66th value
        duplicate_indices.append(65)
    
    # Create a mask to filter out the duplicates
    mask = np.ones(len(data), dtype=bool)
    mask[duplicate_indices] = False
    updated_data = data[mask]

    return updated_data, duplicate_indices



In [ ]:
import numpy as np
from PIL import Image
import os

for k in range(451, 501):
    # Define input and output directories
    input_directory = fr'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\6_test_rx_image\m{k}'  # Folder containing the tiff images
    output_directory= fr'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\7_test_rx_decoded_data\m{k}'  # Folder to save the decoded data

    # Define the batch size
    batch_size = 128  # Adjust the batch size as per your needs

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Get all TIFF files in the input directory
    tiff_files = [f for f in os.listdir(input_directory) if f.endswith('.tiff')]

    # Define the threshold
    threshold = 15.0  # Define threshold for batch separation

    # Loop over all TIFF files in the input directory
    for tiff_file in tiff_files:
        # Load each TIFF image
        image_path = os.path.join(input_directory, tiff_file)
        image = Image.open(image_path)
        image_data = np.array(image).astype(float)

        # Extract specific column range (e.g., columns 850 to 1000)
        columns_range_data = image_data[:, 800:1100]
        column_avg = np.mean(columns_range_data, axis=1)

        # Flatten the array of column-averaged values
        flat_column_avg = column_avg.flatten()
        
        # Initialize variables for batch creation
        batches_column_avg = []
        current_batch_avg = []
        inside_batch_avg = False

        # Create batches based on pixel intensity
        for avg_pixel in flat_column_avg:
            if avg_pixel <= threshold:
                # If inside a batch and value <= threshold, finalize the current batch
                if inside_batch_avg and current_batch_avg:
                    batches_column_avg.append(current_batch_avg)
                    current_batch_avg = []
                inside_batch_avg = True  # Start a new batch when <= threshold encountered
            else:
                if inside_batch_avg:
                    current_batch_avg.append(avg_pixel)

        # Append the last batch if not empty
        if current_batch_avg:
            batches_column_avg.append(current_batch_avg)

        # Process each batch: Average every two pixels while ensuring the last pixel is removed if the averaged batch size isn't divisible by 16
        averaged_batches = []

        # Process each batch and average every two pixels, excluding first and last batch
        for i, batch in enumerate(batches_column_avg[0:-1], start=0):  # Exclude first and last batch
            #print(f"\nProcessing file: {tiff_file}, Batch {i}: Original Data{np.array(batch).shape}")
            #print(np.array(batch))  # Print the original batch content
            updated_data, removed_indices = remove_65(batch)
            # Averaging every two pixels
            averaged_batch = []
            for j in range(0, len(updated_data), 1):
                # Ensure there is a pair to average
                avg_of_two = (updated_data[j])
                averaged_batch.append(avg_of_two)

            # Remove the last pixel if the averaged batch size is not a multiple of 16
            if len(averaged_batch)>128:
                if len(averaged_batch) % 8 != 0: 
                    averaged_batch = averaged_batch[:-(len(averaged_batch) % 8)]

            # Round the averaged batch to 4 decimal places
            averaged_batch = np.round(averaged_batch, 4)

            # Print the averaged batch
            #print(f"Batch {i}: Averaged every two pixels")
            #print(np.array(averaged_batch))  # Print the averaged values
            #print(f"Shape of Averaged Batch {i}: {np.array(averaged_batch).shape}")

            # Check if the current batch has the required length
            if len(averaged_batch) == batch_size:
                # Convert list to comma-separated string
                formatted_list = ', '.join(map(str, averaged_batch))

                # Save the batch to a text file (e.g., m1.txt, m2.txt, etc.)
                output_file_name = os.path.join(output_directory, f"{os.path.splitext(tiff_file)[0]}m{i}.txt")
                with open(output_file_name, 'w') as file:
                    file.write(f"{formatted_list}")

                # Print confirmation
                #print(f"Batch {i} from file {tiff_file} saved to {output_file_name} with length {len(averaged_batch)}")
            else:
                print(f"Batch {i} skipped, length {len(averaged_batch)} (expected length: {batch_size})")

print("Processing complete for all TIFF files.")

## decode data from all images in a folder and save data as .txt file with proper size (2 px)

In [ ]:
import numpy as np
from PIL import Image
import os

for k in range(426, 446):
    # Define input and output directories
    input_directory = fr'F:\My File\OCC_Nov_11\Dataset_16subcarrier\5_test_rx_image\m{k}'  # Folder containing the tiff images
    output_directory=fr'F:\My File\OCC_Nov_11\Dataset_16subcarrier\6_test_rx_decoded_data\m{k}'  # Folder to save the decoded data

    # Define the batch size
    batch_size = 256  # Adjust the batch size as per your needs

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Get all TIFF files in the input directory
    tiff_files = [f for f in os.listdir(input_directory) if f.endswith('.tiff')]

    # Define the threshold
    threshold = 15.0  # Define threshold for batch separation


    # Loop over all TIFF files in the input directory
    for tiff_file in tiff_files:
        # Load each TIFF image
        image_path = os.path.join(input_directory, tiff_file)
        image = Image.open(image_path)
        image_data = np.array(image).astype(float)

        # Extract specific column range (e.g., columns 850 to 1000)
        columns_range_data = image_data[:, 800:1100]
        column_avg = np.mean(columns_range_data, axis=1)

        # Flatten the array of column-averaged values
        flat_column_avg = column_avg.flatten()
        updated_data, removed_indices = remove_consecutive_duplicates(flat_column_avg)
        # print("Updated Data:", updated_data)

        # Initialize variables for batch creation
        batches_column_avg = []
        current_batch_avg = []
        inside_batch_avg = False

        # Create batches based on pixel intensity
        for avg_pixel in updated_data:
            if avg_pixel <= threshold:
                # If inside a batch and value <= threshold, finalize the current batch
                if inside_batch_avg and current_batch_avg:
                    batches_column_avg.append(current_batch_avg)
                    current_batch_avg = []
                inside_batch_avg = True  # Start a new batch when <= threshold encountered
            else:
                if inside_batch_avg:
                    current_batch_avg.append(avg_pixel)

        # Append the last batch if not empty
        if current_batch_avg:
            batches_column_avg.append(current_batch_avg)

        # Process each batch: Average every two pixels while ensuring the last pixel is removed if the averaged batch size isn't divisible by 16
        averaged_batches = []

        # Process each batch and average every two pixels, excluding first and last batch
        for i, batch in enumerate(batches_column_avg[1:-1], start=1):  # Exclude first and last batch
            #print(f"\nProcessing file: {tiff_file}, Batch {i}: Original Data{np.array(batch).shape}")
            #print(np.array(batch))  # Print the original batch content

            # Averaging every two pixels
            averaged_batch = []
            for j in range(0, len(batch), 1):
                # Ensure there is a pair to average
                avg_of_two = (batch[j])
                averaged_batch.append(avg_of_two)

            # Remove the last pixel if the averaged batch size is not a multiple of 16
            if len(averaged_batch) % 16 != 0:
                averaged_batch = []
                for j in range(1, len(batch), 1):
                    # Ensure there is a pair to average
                    avg_of_two = (batch[j])
                    averaged_batch.append(avg_of_two)
                averaged_batch = averaged_batch[:-(len(averaged_batch) % 16)]

            # Round the averaged batch to 4 decimal places
            averaged_batch = np.round(averaged_batch, 4)

            # Print the averaged batch
            #print(f"Batch {i}: Averaged every two pixels")
            #print(np.array(averaged_batch))  # Print the averaged values
            #print(f"Shape of Averaged Batch {i}: {np.array(averaged_batch).shape}")

            # Check if the current batch has the required length
            if len(averaged_batch) == batch_size:
                # Convert list to comma-separated string
                formatted_list = ', '.join(map(str, averaged_batch))

                # Save the batch to a text file (e.g., m1.txt, m2.txt, etc.)
                output_file_name = os.path.join(output_directory, f"m{os.path.splitext(tiff_file)[0]}{i}.txt")
                with open(output_file_name, 'w') as file:
                    file.write(f"{formatted_list}")

                # Print confirmation
                #print(f"Batch {i} from file {tiff_file} saved to {output_file_name} with length {len(averaged_batch)}")
            else:
                print(f"Batch {i} skipped, length {len(averaged_batch)} (expected length: {batch_size})")

print("Processing complete for all TIFF files.")


In [ ]:
import numpy as np
from PIL import Image
import os

# Define input and output directories
input_directory = r'E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\rx_image_dataset_66khz\m104'  # Folder containing the tiff images
output_directory =r'E:\Minhaz\iot\OCC_sep_5\new_dataset_equalizer_10_24\rx_decoded_data\m104'  # Folder to save the decoded data

# Define the batch size
batch_size = 80  # Adjust the batch size as per your needs

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Get all TIFF files in the input directory
tiff_files = [f for f in os.listdir(input_directory) if f.endswith('.tiff')]

# Define the threshold
threshold = 30.0  # Define threshold for batch separation


# Loop over all TIFF files in the input directory
for tiff_file in tiff_files:
    # Load each TIFF image
    image_path = os.path.join(input_directory, tiff_file)
    image = Image.open(image_path)
    image_data = np.array(image).astype(float)

    # Extract specific column range (e.g., columns 850 to 1000)
    columns_range_data = image_data[:, 850:1000]
    column_avg = np.mean(columns_range_data, axis=1)

    # Flatten the array of column-averaged values
    flat_column_avg = column_avg.flatten()

    # Initialize variables for batch creation
    batches_column_avg = []
    current_batch_avg = []
    inside_batch_avg = False

    # Create batches based on pixel intensity
    for avg_pixel in flat_column_avg:
        if avg_pixel <= threshold:
            # If inside a batch and value <= threshold, finalize the current batch
            if inside_batch_avg and current_batch_avg:
                batches_column_avg.append(current_batch_avg)
                current_batch_avg = []
            inside_batch_avg = True  # Start a new batch when <= threshold encountered
        else:
            if inside_batch_avg:
                current_batch_avg.append(avg_pixel)

    # Append the last batch if not empty
    if current_batch_avg:
        batches_column_avg.append(current_batch_avg)

    # Process each batch: Average every two pixels while ensuring the last pixel is removed if the averaged batch size isn't divisible by 16
    averaged_batches = []

    # Process each batch and average every two pixels, excluding first and last batch
    for i, batch in enumerate(batches_column_avg[1:-1], start=1):  # Exclude first and last batch
        print(f"\nProcessing file: {tiff_file}, Batch {i}: Original Data{np.array(batch).shape}")
        print(np.array(batch))  # Print the original batch content

        # Averaging every two pixels
        averaged_batch = []
        for j in range(0, len(batch), 2):
            # Ensure there is a pair to average
            if j + 1 < len(batch):
                avg_of_two = (batch[j] + batch[j + 1]) / 2
                averaged_batch.append(avg_of_two)

        # Remove the last pixel if the averaged batch size is not a multiple of 16
        if len(averaged_batch) % 16 != 0:
            averaged_batch = averaged_batch[:-(len(averaged_batch) % 16)]

        # Round the averaged batch to 4 decimal places
        averaged_batch = np.round(averaged_batch, 4)

        # Print the averaged batch
        print(f"Batch {i}: Averaged every two pixels")
        print(np.array(averaged_batch))  # Print the averaged values
        print(f"Shape of Averaged Batch {i}: {np.array(averaged_batch).shape}")

        # Check if the current batch has the required length
        if len(averaged_batch) == batch_size:
            # Convert list to comma-separated string
            formatted_list = ', '.join(map(str, averaged_batch))

            # Save the batch to a text file (e.g., m1.txt, m2.txt, etc.)
            output_file_name = os.path.join(output_directory, f"{os.path.splitext(tiff_file)[0]}m{i}.txt")
            with open(output_file_name, 'w') as file:
                file.write(f"{formatted_list}")

            # Print confirmation
            print(f"Batch {i} from file {tiff_file} saved to {output_file_name} with length {len(averaged_batch)}")
        else:
            print(f"Batch {i} skipped, length {len(averaged_batch)} (expected length: {batch_size})")

print("Processing complete for all TIFF files.")


## decode data from images and save data as .txt file with proper size

In [ ]:
import numpy as np
from PIL import Image
import os

# Load the TIFF image
image_path = r'E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\rx_image_dataset_66khz\m39\1.tiff'  # Update with the correct file path / change file name
# Define output directory to save the files
output_directory = r'E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\Rx_decoded_data\m39' # change the folder name

image = Image.open(image_path)
image_data = np.array(image).astype(float)

# Extract specific column range (951 to 1000)
columns_range_data = image_data[:, 850:1050]
column_avg = np.mean(columns_range_data, axis=1)

# Flatten the array of column-averaged values
flat_column_avg = column_avg.flatten()

# Initialize variables for batch creation
batches_column_avg = []
current_batch_avg = []
threshold = 20.0  # Define threshold for batch separation
inside_batch_avg = False

# Create batches based on pixel intensity
for avg_pixel in flat_column_avg:
    if avg_pixel <= threshold:
        # If inside a batch and value <= threshold, finalize the current batch
        if inside_batch_avg and current_batch_avg:
            batches_column_avg.append(current_batch_avg)
            current_batch_avg = []
        inside_batch_avg = True  # Start a new batch when <= threshold encountered
    else:
        if inside_batch_avg:
            current_batch_avg.append(avg_pixel)

# Append the last batch if not empty
if current_batch_avg:
    batches_column_avg.append(current_batch_avg)

# Process each batch: Average every two pixels while ensuring the last pixel is removed if the averaged batch size isn't divisible by 16
averaged_batches = []

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Process each batch and average every two pixels, excluding first and last batch
for i, batch in enumerate(batches_column_avg[1:-1], start=1):  # Exclude first and last batch
    print(f"\nBatch {i}: Original Data{np.array(batch).shape}")
    print(np.array(batch))  # Print the original batch content

    # Averaging every two pixels
    averaged_batch = []
    for j in range(0, len(batch), 2):
        # Ensure there is a pair to average
        if j + 1 < len(batch):
            avg_of_two = (batch[j] + batch[j + 1]) / 2
            averaged_batch.append(avg_of_two)

    # Remove the last pixel if the averaged batch size is not a multiple of 16
    if len(averaged_batch) % 16 != 0:
        averaged_batch = averaged_batch[:-(len(averaged_batch) % 16)]

    # Round the averaged batch to 4 decimal places
    averaged_batch = np.round(averaged_batch, 4)

    # Print the averaged batch
    print(f"Batch {i}: Averaged every two pixels")
    print(np.array(averaged_batch))  # Print the averaged values
    print(f"Shape of Averaged Batch {i}: {np.array(averaged_batch).shape}")

    # Convert the list to a format with commas
    formatted_list = ', '.join(map(str, averaged_batch))

    # Check if the current batch has the maximum length
    if len(averaged_batch) == 160:  # change batch size
        # Convert list to comma-separated string
        formatted_list = ', '.join(map(str, averaged_batch))

        # Save the batch to a text file named m1.txt, m2.txt, etc.
        file_name = os.path.join(output_directory, f"mi1{i}.txt")
        with open(file_name, 'w') as file:
            file.write(f"{formatted_list}")
        
        # Print confirmation
        print(f"Batch {i} saved to {file_name} with length {len(averaged_batch)}")
    else:
        print(f"Batch {i} skipped, length {len(averaged_batch)} (max length is 128)")

# Optionally, you could visualize the batches or perform further analysis
# This would depend on how you wish to analyze or present the data


## rename files in a folder

In [ ]:
# import os

# # Define the directory path
# directory = r"E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\rx_image_dataset_66khz\m10"

# # List all files in the directory
# files = sorted(os.listdir(directory))

# # Loop over the files and rename them
# for index, filename in enumerate(files):
#     if filename.endswith(".tiff"):  # Ensure the file is a tiff image
#         new_name = f"{index + 1}.tiff"  # New file name
#         old_file = os.path.join(directory, filename)
#         new_file = os.path.join(directory, new_name)
#         os.rename(old_file, new_file)
#         print(f"Renamed {filename} to {new_name}")


## rename files in the folders

In [ ]:
import os

# Base directory path
base_directory = fr"F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\3_rx_image"

# Loop through folders m1 to m425
for folder_number in range(1, 506):
    directory = os.path.join(base_directory, f"m{folder_number}")
    
    # List all files in the directory
    files = sorted(os.listdir(directory))
    
    # Loop over the files and rename them
    for index, filename in enumerate(files):
        if filename.endswith(".tiff"):  # Ensure the file is a tiff image
            new_name = f"{index + 1}.tiff"  # New file name
            old_file = os.path.join(directory, filename)
            new_file = os.path.join(directory, new_name)
            os.rename(old_file, new_file)
            print(f"Renamed {filename} in folder m{folder_number} to {new_name}")
